# Get Initial Markers
Before running the tracking algorithm on the video, you need to define the initial markers of the objects. You have to do this based on the first frame. The markers do not have to be exactly in the middle of the object, but approximately the middle is good enough (the algorithm will correct this for you). You can specify which video to consider below:

In [ ]:
from IPython import display
import os

import cv2
import scipy as sp
import skimage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import yaml

from fam13a import consts, utils, track, image, kalman

In [ ]:
PROJ_ROOT = utils.here(True)
VIDEO_ROOT = os.path.join(PROJ_ROOT, 'data', 'interim', 'xenopus')
MARKERS_SEGMENTED_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'xenopus', 'segmented', 'markers')


In [ ]:
#Print all videos where markers have already been generated.
print(sorted([_f for _f in os.listdir(MARKERS_SEGMENTED_ROOT) if not _f.endswith('dvc')]))

In [ ]:
# Print all available videos
GRAD_SEGMENTED_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'xenopus', 'segmented', 'gradient')
COLOUR_SEGMENTED_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'xenopus', 'segmented', 'colour')
print(sorted([_f for _f in os.listdir(GRAD_SEGMENTED_ROOT) if not _f.endswith('dvc')]))

In [ ]:
VIDEO_ID = '5_L2_1'
raw_frames = utils.frames_from_video(os.path.join(VIDEO_ROOT, f'{VIDEO_ID}.mp4'))
frame = cv2.cvtColor(raw_frames[0, :],
                     cv2.COLOR_BGR2HSV)

# This will show the first frame, from which you can derive the locations of the markers.

In [ ]:
fig = plt.figure()
fig.set_size_inches(10,10)
plt.imshow(frame)
minorLocator = MultipleLocator(100)
plt.grid(b=True, which="both")
plt.gca().xaxis.set_minor_locator(minorLocator)
plt.gca().yaxis.set_minor_locator(minorLocator)
plt.show()

# Create initial set of markers using `image.detect.centers_of_mass`

In [ ]:
grad_frames = os.listdir(os.path.join(GRAD_SEGMENTED_ROOT, VIDEO_ID))
grad_frames = sorted(grad_frames)
grad_frames = np.load(os.path.join(GRAD_SEGMENTED_ROOT, VIDEO_ID, grad_frames[0]))
grad_frames.shape

In [ ]:
colour_frames = os.listdir(os.path.join(COLOUR_SEGMENTED_ROOT, VIDEO_ID))
colour_frames = sorted(colour_frames)
colour_frames = np.load(os.path.join(COLOUR_SEGMENTED_ROOT, VIDEO_ID, colour_frames[0]))
colour_frames.shape

In [ ]:
comb_frames = np.logical_and(colour_frames, grad_frames).astype(np.uint8)
comb_frames = image.utils.filter_area(comb_frames, min_area=1000)

In [ ]:
# Centres of mass for each segmented object
points = image.detect.centers_of_mass(sp.ndimage.label(comb_frames)[0]) 
points = {k.item():list(v) for k,v in points.items()}


# Evaluate initial markers
Run this after you have created the file with markers

In [ ]:
# If you want to look at what markers are in the yml file, set show_stored_markers to True
show_stored_markers = False
if show_stored_markers:
    with open(os.path.join(MARKERS_SEGMENTED_ROOT, VIDEO_ID + ".yml"), "r") as f:
        points = yaml.load(f, Loader=yaml.SafeLoader)

In [ ]:
fig = plt.figure()
fig.set_size_inches(10,10)
plt.imshow(frame)
plt.scatter([value[1] for key, value in points.items()], [value[0] for key, value in points.items()], color="r", marker="*")
minorLocator = MultipleLocator(100)
plt.grid(b=True, which="both")
plt.gca().xaxis.set_minor_locator(minorLocator)
plt.gca().yaxis.set_minor_locator(minorLocator)
plt.show()

In [ ]:
# Display the points
points

In [ ]:
## Append additional points
# points[14] = [320, 175] # Comment in this line and change as appropriate. Also append more points as appropriate

In [ ]:
## shifting all points by a constant amount in a specific direction
#points = {k: [v[0]+50, v[1]] for k,v in points.items()} # This line can be used to shift all points (e.g. by 50)

# Defining markers
The points dictionary has an integer as key and a list of coordinates as values. Append points as indicated in the line above.
Note than when looking at the plot, the axes are flipped, so what looks like an X coordinate in the image is Y and vice versa. 

In [ ]:
# Check the new points have been added
points

In [ ]:
fig = plt.figure()
fig.set_size_inches(10,10)
plt.imshow(frame)
plt.scatter([value[1] for key, value in points.items()], [value[0] for key, value in points.items()], color="r", marker="*")
minorLocator = MultipleLocator(100)
plt.grid(b=True, which="both")
plt.gca().xaxis.set_minor_locator(minorLocator)
plt.gca().yaxis.set_minor_locator(minorLocator)
plt.show()

### Once you are happy with the markers, store them in the yml file

In [ ]:
with open(os.path.join(MARKERS_SEGMENTED_ROOT, VIDEO_ID + ".yml"), 'w') as outfile:
    yaml.dump(points, outfile, default_flow_style = False)
print("Markers for this video are stored at: " + os.path.join(MARKERS_SEGMENTED_ROOT, VIDEO_ID + ".yml"))